http://www.cs.cmu.edu/~ark/personas/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MultiLabelBinarizer
%config InlineBackend.figure_format = 'retina'

In [2]:
genres = pd.read_csv(
    'data/raw/movie.metadata.tsv',
    converters={'genres': lambda x: list(eval(x).values())},
    delimiter='\t',
    header=None,
    index_col='id',
    names=['id', 'genres'],
    usecols=[0, 8])

summaries = pd.read_csv(
    'data/raw/plot_summaries.txt',
    delimiter='\t',
    header=None,
    index_col='id',
    names=['id', 'summary'])

df = summaries.merge(genres, on='id').reset_index(drop=True)
mlb = MultiLabelBinarizer()
labels = pd.DataFrame(mlb.fit_transform(df.genres), columns=mlb.classes_)
X, Y = df[['summary']], labels[labels.sum().nlargest(10).sort_index().index]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=10000, random_state=0)

In [3]:
make_pipeline(
    make_column_transformer((TfidfVectorizer(), 'summary')),
    MultiOutputClassifier(LogisticRegression(solver='liblinear', random_state=0), n_jobs=-1)
).fit(X_train, Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('tfidfvectorizer',
                                                  TfidfVectorizer(),
                                                  'summary')])),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LogisticRegression(random_state=0,
                                                                    solver='liblinear'),
                                       n_jobs=-1))])